In [3]:
# upload dataset

import json

file_path = r"C:\Users\LENOVO\OneDrive\Desktop\Desktop\Mikro ChatBot\Mikro_LSTM_Dataset.json"
with open(file_path, 'r', encoding='utf-8') as f:
    mikro_data = json.load(f)

In [2]:
# Importing the modules and libraries
import nltk
import numpy as np
import json
import pickle
import random

from nltk.stem import WordNetLemmatizer
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding,LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam

# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Load Dataset
with open('Mikro_LSTM_Dataset.json','r') as file:
    data=json.load(file)

In [4]:
lemmatizer = WordNetLemmatizer()
sentences, labels = [], []
classes = []
ignore_chars = ['?', '!', ',', '.']

In [5]:
# Process dataset
for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words = [lemmatizer.lemmatize(w.lower()) for w in word_list if w not in ignore_chars]
        sentences.append(" ".join(words))
        labels.append(intent['tag'])
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

classes = sorted(set(classes))

In [6]:
# Tokenize sentences
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word_index = tokenizer.word_index

In [7]:
# Pad sequences to the same length
max_length = max(len(seq) for seq in sequences)
x_train = pad_sequences(sequences, maxlen=max_length, padding='post')


In [8]:
# Convert labels to numerical form
label_dict = {label: index for index, label in enumerate(classes)}
y_train = np.array([label_dict[label] for label in labels])

In [9]:
# Save tokenizer and classes
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [10]:
# Build LSTM(Long-Short Term Memory) model with Embedding (Keras 3 Fix)
model = Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=128, mask_zero=True),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(classes), activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [12]:
# Train model
model.fit(x_train, y_train, epochs=100, batch_size=8, verbose=1)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 36ms/step - accuracy: 0.0213 - loss: 4.0269
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.0830 - loss: 4.0150
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.0891 - loss: 3.9839
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.0871 - loss: 3.7719
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.1586 - loss: 3.3986
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.1448 - loss: 3.0578
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.2629 - loss: 2.6620
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.3248 - loss: 2.3175
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4267 - loss: 1.9316
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.6150 - loss: 1.6926
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.5538 - loss: 1.5468
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/ste

In [52]:
model.save('chatbot_model.keras')  # Save in Keras 3 format

In [14]:
def chatbot_response(text):
    words = [lemmatizer.lemmatize(w.lower()) for w in nltk.word_tokenize(text)]
    seq = tokenizer.texts_to_sequences([" ".join(words)])
    padded_seq = pad_sequences(seq, maxlen=max_length,padding='post')

    prediction = model.predict(padded_seq,verbose=0)[0]
    tag = classes[np.argmax(prediction)]

    for intent in data['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    return "I'm not sure about that. Can you ask something else?"

In [15]:
if __name__ == "__main__":
    print("Chatbot is ready! Type 'exit' to stop.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Thank you for using Airbus chatbot tool. Goodbye!")
            break
        print("Bot: ",chatbot_response(user_input))

Chatbot is ready! Type 'exit' to stop.
You: exit
Thank you for using Airbus chatbot tool. Goodbye!


In [1]:
%pip install streamlit --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\LENOVO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [49]:
%%writefile app.py
import streamlit as st
import nltk
import numpy as np
import json
import pickle
import random

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')
nltk.download('wordnet')

# Initialize tools
lemmatizer = WordNetLemmatizer()

# Load model and preprocessing objects
model = load_model('chatbot_model.keras')
tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Load intents
with open('dataset.json', 'r') as file:
    data = json.load(file)

# Max input length
max_length = model.input_shape[1]

# Chatbot response function
def chatbot_response(text):
    words = [lemmatizer.lemmatize(w.lower()) for w in nltk.word_tokenize(text)]
    seq = tokenizer.texts_to_sequences([" ".join(words)])
    padded_seq = pad_sequences(seq, maxlen=max_length, padding='post')
    prediction = model.predict(padded_seq, verbose=0)[0]
    tag = classes[np.argmax(prediction)]

    for intent in data['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    return "I'm not sure about that. Can you ask something else?"

# Page setup
st.set_page_config(layout="wide", page_title="Mikro's Chatbot", page_icon="🤖")

if "messages" not in st.session_state:
    st.session_state.messages = []

# Sidebar settings
with st.sidebar:
    st.title("⚙️")
    st.markdown("Welcome to the **Mikro's Chatbot**. Ask questions about defense, anatomy, or any trained topic.")
    if st.button("Clear Chat"):
        st.session_state.messages = []
        st.rerun()

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Main layout
col1, col2, col3 = st.columns([3, 0.2, 1])




# Left side: chat
with col1:

  st.markdown("""
    <div style="padding-top: 100px;">
        <h2>🤖 Mikro's Chatbot</h2>
        <h4>Ask me anything related to your dataset!</h4>
    </div>
""", unsafe_allow_html=True)
# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display past messages
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Chat input
prompt = st.chat_input("Type your message...")

if prompt:
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # Generate response
    response = chatbot_response(prompt)

    # Add bot message
    st.session_state.messages.append({"role": "assistant", "content": response})
    with st.chat_message("assistant"):
        st.markdown(response)



# Right side: info panel
with col3:
    st.markdown("### 📌 Info Panel")
    st.markdown("- Supports Airbus, Defense & Human Anatomy topics")
    st.markdown("- Uses trained Keras model")
    st.markdown("- Powered by Streamlit UI")
    st.markdown("- Built for learning & demo")

Overwriting app.py


In [4]:
import toml

config = r"C:\Users\LENOVO\OneDrive\Desktop\Desktop\Mikro ChatBot\config.toml"
with open(config, 'r', encoding='utf-8') as f:
    config_data = toml.load(f)

In [5]:
import streamlit as st
import toml

# Load config
with open('config.toml', 'r') as f:
    config = toml.load(f)
# Apply theme settings from config
st.markdown(
    f"""
    <style>
        .stApp {{
            background-color: {config['theme']['backgroundColor']};
            color: {config['theme']['textColor']};
            font-family: {config['theme']['font']};
        }}
        .stButton>button {{
            background-color: {config['theme']['primaryColor']};
        }}
    </style>
    """,
    unsafe_allow_html=True
)

2025-04-23 07:55:29.327 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 07:55:29.808 
  command:

    streamlit run C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-23 07:55:29.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
%streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.74.118.1:8501

⠼⠴⠦your url is: https://swift-moons-laugh.loca.lt
2025-04-22 10:11:50.175898: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745316710.205471   21619 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745316710.214097   21619 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is alre